In [1]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# input_text = "translate English to German: How old are you?"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))

In [13]:

import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [14]:
path = "D:/ABSA work/codes/processed_"
data = "MAMS/"

In [15]:
df_train= pd.read_csv(path+data+"ACSA_train_single.csv")
df_val = pd.read_csv(path+data+"ACSA_val_single.csv")
df_test = pd.read_csv(path+data+"ACSA_test_single.csv")

# Prepare Train Dataset


## Train Data

In [17]:


# Convert string polarities to numerical values
polarity_mapping = {"negative": 0, "neutral": 1, "positive": 2}
df_train['polarity'] = df_train['polarity'].map(polarity_mapping)
print(df_train['polarity'].shape)
# Extract sentences, aspects, and polarities from DataFrame columns
sentences = df_train['sentence'].tolist()
aspects = df_train['aspect_category'].tolist()
polarities = df_train['polarity'].tolist()

input_ids = []
attention_masks = []

for sentence, aspect in zip(sentences, aspects):
    encoded = tokenizer.encode_plus(
        sentence,
        aspect,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
polarities = torch.tensor(polarities)

train_dataset = TensorDataset(input_ids, attention_masks, polarities)

train_size = len(df_train)




(7090,)


c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## Val Data

In [18]:


# Convert string polarities to numerical values
df_val['polarity'] = df_val['polarity'].map(polarity_mapping)
print(df_val['polarity'].shape)
# Extract sentences, aspects, and polarities from DataFrame columns
sentences = df_val['sentence'].tolist()
aspects = df_val['aspect_category'].tolist()
polarities = df_val['polarity'].tolist()

input_ids = []
attention_masks = []

for sentence, aspect in zip(sentences, aspects):
    encoded = tokenizer.encode_plus(
        sentence,
        aspect,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
polarities = torch.tensor(polarities)

val_dataset = TensorDataset(input_ids, attention_masks, polarities)

val_size = len(df_val)
# val_size = (len(dataset) - train_size) // 2
# test_size = len(dataset) - train_size - val_size

# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

(888,)


c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(



## Test Data

In [19]:


# Convert string polarities to numerical values

df_test['polarity'] = df_test['polarity'].map(polarity_mapping)
print(df_test['polarity'].shape)
# Extract sentences, aspects, and polarities from DataFrame columns
sentences = df_test['sentence'].tolist()
aspects = df_test['aspect_category'].tolist()
polarities = df_test['polarity'].tolist()

input_ids = []
attention_masks = []

for sentence, aspect in zip(sentences, aspects):
    encoded = tokenizer.encode_plus(
        sentence,
        aspect,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
polarities = torch.tensor(polarities)

test_dataset = TensorDataset(input_ids, attention_masks, polarities)


test_size = len(df_test)
# val_size = (len(dataset) - train_size) // 2
# test_size = len(dataset) - train_size - val_size

# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])


(901,)


c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# Data Loaders

In [26]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Load Model

In [29]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AdamW

model = AutoModelForSeq2SeqLM.from_pretrained("shorthillsai/flan-t5-large-absa", device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained("shorthillsai/flan-t5-large-absa", truncation=True)

prompt = """Find the aspect based sentiment for the given review. 'Not present' if the aspect is absent.\n\nReview:I love the screen of this laptop and the battery life is amazing.\n\nAspect:Battery Life\n\nSentiment: """

# input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
# instruct_model_outputs = model.generate(input_ids=input_ids)
# instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 8.00 GiB total capacity; 7.26 GiB already allocated; 0 bytes free; 7.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
# Training loop
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 20
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    print(f'Epoch [{epoch+1}/{num_epochs}],')
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        # print(print(f'Epoch [{epoch+1}/{num_epochs}], Batch: {loss}'))
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_loss /= len(val_loader)
    val_acc = correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

Epoch [1/20],


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

# Load Dataset

In [98]:

path = "flan_t5/results/"
data_path="MAMS/"
filename="ACSA_test"
if(os.path.isfile(path+data_path+filename+"_t5.csv")):
    res = pd.read_csv(path+data_path+filename+"_t5.csv")
else:
    res = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])
print(len(res))
res.head(3)

901


,dataset_name,sentence,aspect,polarity,sentiment_t5
0,MAMS/ACSA_test,"We went again and sat at the bar this time, I ...",place,neutral,Negative
1,MAMS/ACSA_test,"We went again and sat at the bar this time, I ...",food,negative,Negative
2,MAMS/ACSA_test,"The food was good, but it's not worth the wait...",food,positive,Neutral


In [99]:
import pandas as pd

df = pd.read_csv('processed_'+data_path+filename+'_single.csv')
# df = pd.read_csv('processed_'+data_path+filename+'_single.csv')
print(len(df))
df.head(2)

901


,sentence,aspect_category,polarity
0,"We went again and sat at the bar this time, I ...",place,neutral
1,"We went again and sat at the bar this time, I ...",food,negative


# Iterate each Row

In [100]:
df.dropna(subset=['aspect_category'], inplace=True)
print(len(df), len(res))
# for sentence, aspec, senti in zip(df['sentence'][len(res):1],df['aspect_term'][len(res):1], df['polarity'][len(res):1]):
for sentence, aspec, senti in zip(df['sentence'][len(res):],df['aspect_category'][len(res):], df['polarity'][len(res):]):
    temp = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])   
    prompt = f"""Review: {sentence}
    Aspect List: {aspec.replace('#', ' ').lower()}
    Predict the sentiment for mentioned aspect in the review. Assign one of the following sentiment to the aspect: Positive , Negative , Neutral.
    Please provide the sentiment only. No any explanation or extra text"""
    # print(prompt)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
    instruct_model_outputs = model.generate(input_ids=input_ids)
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    data_to_append = {
    'dataset_name': data_path+filename,
    'sentence': sentence,
    'aspect': aspec,
    'polarity': senti,
    'sentiment_t5': instruct_model_text_output 
    }

    print(data_to_append)
    temp = temp.append(data_to_append, ignore_index=True)
    file_path = path+data_path+filename+"_t5.csv"
    if os.path.exists(file_path):   
        temp.to_csv(file_path, index=False, mode='a', header=False)
    else:
        temp.to_csv(file_path, index=False, mode='a')
    

901 901


In [38]:
# #Old Prompt
# for sentence, aspec, senti in zip(df['sentence'][len(res):1],df['aspect_category'][len(res):1], df['polarity'][len(res):1]):
# # for sentence, aspec, senti in zip(df['sentence'][len(res):],df['aspect_category'][len(res):], df['polarity'][len(res):]):
#     temp = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])
#     prompt = f"""Find the aspect based sentiment for the given review. 'Not present' if the aspect is absent.
#     \n\nReview: {sentence.lower()}.
#     \n\nAspect: {aspec.replace('#', ' ').lower()} \n\nSentiment: """
#     # print(prompt)
#     input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
#     instruct_model_outputs = model.generate(input_ids=input_ids)
#     instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

#     data_to_append = {
#     'dataset_name': data_path+filename,
#     'sentence': sentence,
#     'aspect': aspec,
#     'polarity': senti,
#     'sentiment_t5': instruct_model_text_output 
#     }

#     print(data_to_append)
#     temp = temp.append(data_to_append, ignore_index=True)
#     temp.to_csv(path+data_path+filename+"_category_t5.csv", index=False, mode='a')

In [38]:
%pwd

'd:\\ABSA work\\codes'